# 基于寒武纪 MLU 的模型训练--YOLOv5目标检测
### --PyTorch, Python3, FP32

## 目录
### 0 基本信息
### 1 实验内容及目标
     1.1 实验内容
     1.2 实验目标
### 2 前置知识介绍
     2.1 寒武纪软硬件平台
     2.2 寒武纪 PyTorch 框架
### 3 网络详解
     3.1 网络结构
### 4 模型训练
     4.1 工程目录介绍
     4.2 工程准备
     4.3 移植修改
     4.4 训练
     4.5 精度验证
### 5 结语
     5.1 回顾重点步骤
     5.2 相关链接


# 0 基本信息

发布者：寒武纪

实验时长：120 分钟

语言：Python3

修改时间：2022-10-12

# 1 实验内容及目标
## 1.1 实验内容

本实验主要介绍基于寒武纪 MLU370 (寒武纪处理器，简称 MLU )与寒武纪 PyTorch 框架的 YOLOv5（v6.0版本）目标检测训练方法。在官方源码的基础上，只需要进行简单移植和修便可在 MLU370 加速训练 YOLOv5 算法，实现目标检测的功能。后续章节将会详细介绍移植过程。


## 1.2 实验目标

1. 掌握使用寒武纪 MLU370 和 PyTorch 框架进行 AI 模型训练的基本方法。

2. 理解 YOLOv5 模型的整体网络结构及其适配流程。


# 2 前置知识介绍

## 2.1 寒武纪软硬件平台介绍

 &emsp; 硬件：寒武纪 MLU370 AI 计算卡 
 
 &emsp; 框架：PyTorch 1.9 
 
 &emsp; 系统环境：寒武纪云平台 

## 2.2 寒武纪 PyTorch 框架
为⽀持寒武纪 MLU 加速卡，寒武纪定制了开源⼈⼯智能编程框架PyTorch（以下简称 Cambricon PyTorch）。    

Cambricon PyTorch 借助 PyTorch ⾃⾝提供的设备扩展接⼝将 MLU 后端库中所包含的算⼦操作动态注册到 PyTorch 中，MLU 的后端库可处理 MLU 上的张量和⽹络算⼦的运算。Cambricon PyTorch 会基于 CNNL 库在 MLU 后端实现常⽤⽹络算⼦的计算，并完成数据拷⻉。    

Cambricon PyTorch 兼容原⽣ PyTorch 的 Python 编程接⼝和原⽣ PyTorch ⽹络模型，⽀持以在线逐层⽅式进⾏训练和推理。⽹络模型可以从 pth 或 pt 格式⽂件读取，已⽀持的分类和检测⽹络结构由 Torchvision管理，可以从 Torchvision 中读取。对于训练任务，⽀持 float32 及定点量化模型。  

获取更多有关Cambricon PyTorch资料，请参考 [寒武纪官网文档](https://developer.cambricon.com/index/document/index/classid/3.html)  PyTorch相关内容。  

# 3 模型架构

## 网络结构

YOLOv5针对不同大小（n,s,m,l,x）的网络整体架构不变，但会根据yaml文件中定义的 **depth_mutiple**和**width_mutiple** 参数，对每个子模块进行不同深度和宽度的缩放。

以[YOLOv5m](https://github.com/ultralytics/yolov5/blob/v6.1/models/yolov5m.yaml)为例，其网络结构主要由以下几部分组成:

```depth_multiple: 0.67 ``` 深度扩充：max(round(n * depth_multiple),1)，其中 **n** 为yaml文件中的 **number** 参数。

```width_multiple: 0.75 ``` 宽度（通道）扩充：ceil(width_multiple \* args\[0\] / 8 ) * 8，作用于yaml文件中的**args\[0\]**即卷积输出通道数。

* Backbone: Conv(CBS), C3, SPPF。 

```
[from, number, module, args] 
[[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2 
[-1, 1, Conv, [128, 3, 2]],  # 1-P2/4 
[-1, 3, C3, [128]],  
[-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
[-1, 6, C3, [256]], \
[-1, 1, Conv, [512, 3, 2]],  # 5-P4/16 
[-1, 9, C3, [512]], \
[-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32 
[-1, 3, C3, [1024]], \
[-1, 1, SPPF, [1024, 5]],  # 9 
] 
```

* Head: Conv(CBS), C3, Upsample, Concat。

```
[[-1, 1, Conv, [512, 1, 1]], 
[-1, 1, nn.Upsample, [None, 2, 'nearest']], 
[[-1, 6], 1, Concat, [1]],  # cat backbone P4 
[-1, 3, C3, [512, False]],  # 13 

[-1, 1, Conv, [256, 1, 1]], 
[-1, 1, nn.Upsample, [None, 2, 'nearest']], 
[[-1, 4], 1, Concat, [1]],  # cat backbone P3 
[-1, 3, C3, [256, False]],  # 17 (P3/8-small) 
 
[-1, 1, Conv, [256, 3, 2]], 
[[-1, 14], 1, Concat, [1]],  # cat head P4 
[-1, 3, C3, [512, False]],  # 20 (P4/16-medium) 

[-1, 1, Conv, [512, 3, 2]], 
[[-1, 10], 1, Concat, [1]],  # cat head P5 
[-1, 3, C3, [1024, False]],  # 23 (P5/32-large) 

[[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5) 
]
```

* Detect: Decode, nms, topk(CPU端运行，不参与训练)。

网络结构如下图所示:

![avatar](./course_images/yolov5m.png)

 其中:
 
* CBS: 由 Conv + BN2d + SiLU 三者组成。
 
* C3: [CSP Bottleneck](https://github.com/WongKinYiu/CrossStagePartialNetworks) with 3 convolutions,结构图上图所示。
 
* SPPF: Spatial Pyramid Pooling (Fast)，空间金字塔池化结构，功能不变前提下提升运行速度。

# 4 模型训练

## 4.1 工程目录介绍
```
└── pytorch_yolov5_train
    ├── apply_patch.sh           
    ├── course_images
    │   └── yolov5m.png
    ├── pytorch_yolov5_train.ipynb
    ├── README.md   
    ├── yolov5                        #  YOLOv5 工程，切换到v6.0版本
    ├── requirements_mlu.txt
    ├── prepare.sh
    ├── utils_mlu
    │   ├── collect_env.py
    │   ├── common_utils.sh
    │   ├── configs.json
    │   ├── metric.py
    │   └── __pycache__
    └── yolov5_mlu.patch
    ├── yolov5_model
        └── yolov5m.yaml

```

## 4.2 工程准备


1. 安装依赖

In [ ]:
!pip install -r ./requirements_mlu.txt

In [ ]:
!apt update
!apt install -y curl 

2. 模型和数据集下载

默认使用 COCO 2017 数据集进行训练，这里数据集存放路径与官方一致，可直接使用官方脚本下载数据集。 
```
Download COCO 2017 dataset http://cocodataset.org
Example usage: bash data/scripts/get_coco.sh \
practices             # 实验平台是 workspace
├── projects 
├── model 
     └── pretrained  
          └── coco  
└── dataset 
     └── private   
          └── coco  ← downloads here
```

In [ ]:
!sh prepare.sh

## 4.3 移植修改

本实验实在官网原始 YOLOv5 工程的 v6.0 版本下移植修改而得到的。为便于用户移植和修改，我们通过 patch 方式将适配后的训练代码应用于源码。patch代码见 [yolov5_mlu.patch](./yolov5_mlu.patch)  命令如下：

In [ ]:
!bash apply_patch.sh

下面，我们将根据**yolov5_mlu.patch**对训练代码适配MLU370过程进行详细解析。

* **train.py**首先以train.py进行分析，其patch内容如下：

```
diff --git a/train.py b/train.py
index 29ae43e..e56b1d5 100644
--- a/train.py
+++ b/train.py
@@ -44,18 +44,22 @@ from utils.downloads import attempt_download
 from utils.loss import ComputeLoss
 from utils.plots import plot_labels, plot_evolve
 from utils.torch_utils import EarlyStopping, ModelEMA, de_parallel, intersect_dicts, select_device, \
-    torch_distributed_zero_first
+    torch_distributed_zero_first, is_device_available
 from utils.loggers.wandb.wandb_utils import check_wandb_resume
 from utils.metrics import fitness
 from utils.loggers import Loggers
 from utils.callbacks import Callbacks
 
+# benchmark
+cur_dir = os.path.dirname(os.path.abspath(__file__))
+sys.path.append(cur_dir + "/../utils_mlu/")
+from metric import MetricCollector
+
```

从上述代码可知，主要改动为：

1. 在utils/torch_utils.py文件中新增**is_device_available**函数,该函数可判断mlu_device是否可获取，如下所示：

```
diff --git a/utils/torch_utils.py b/utils/torch_utils.py
index 352ecf5..fd97b42 100644
--- a/utils/torch_utils.py
+++ b/utils/torch_utils.py
@@ -27,6 +27,10 @@ except ImportError:
 
 LOGGER = logging.getLogger(__name__)
 
+is_device_available = {
+        'cuda': torch.cuda.is_available(),
+        'mlu' : hasattr(torch, 'is_mlu_available') and torch.is_mlu_available()
+    }
```
2. 新增导入utils_mlu目录，主要用于benchmark性能数据测试收集；

---
```
@@ -97,7 +101,7 @@ def train(hyp,  # path/to/hyp.yaml or hyp dictionary

     # Config
     plots = not evolve  # create plots
-    cuda = device.type != 'cpu'
+    use_device = device.type in ['cuda', 'mlu']
     init_seeds(1 + RANK)
     with torch_distributed_zero_first(LOCAL_RANK):
         data_dict = data_dict or check_dataset(data)  # check if None
@@ -113,7 +117,7 @@ def train(hyp,  # path/to/hyp.yaml or hyp dictionary
     if pretrained:
         with torch_distributed_zero_first(LOCAL_RANK):
             weights = attempt_download(weights)  # download if not found locally
-        ckpt = torch.load(weights, map_location=device)  # load checkpoint
+        ckpt = torch.load(weights, map_location='cpu')  # load checkpoint
         model = Model(cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
         exclude = ['anchor'] if (cfg or hyp.get('anchors')) and not resume else []  # exclude keys
         csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
```
解析：
1. ```device.type```支持了‘mlu’选项，表示用mlu进行训练；
2. 运行到pertrained分支，即采用预训练模型时，此时load模型文件要从cpu端进行获取，即```map_location='cpu'```
---
```
@@ -196,13 +200,14 @@ def train(hyp,  # path/to/hyp.yaml or hyp dictionary
     imgsz = check_img_size(opt.imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

     # DP mode
-    if cuda and RANK == -1 and torch.cuda.device_count() > 1:
+    cuda = (device.type=='cuda')
+    if cuda and RANK == -1 and torch.cuda.device_count()  > 1:
         logging.warning('DP not recommended, instead use torch.distributed.run for best DDP Multi-GPU results.\n'
                         'See Multi-GPU Tutorial at https://github.com/ultralytics/yolov5/issues/475 to get started.')
         model = torch.nn.DataParallel(model)

     # SyncBatchNorm
-    if opt.sync_bn and cuda and RANK != -1:
+    if opt.sync_bn and use_device and RANK != -1:
         model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)
         LOGGER.info('Using SyncBatchNorm()')

@@ -238,7 +243,7 @@ def train(hyp,  # path/to/hyp.yaml or hyp dictionary
         callbacks.run('on_pretrain_routine_end')

     # DDP mode
-    if cuda and RANK != -1:
+    if use_device and RANK != -1:
         model = DDP(model, device_ids=[LOCAL_RANK], output_device=LOCAL_RANK)
```

解析：上述修改涉及于分布式训练，这里暂不对此进行介绍。详细内容可以参考请参考[寒武纪官网文档](https://developer.cambricon.com/index/document/index/classid/3.html) PyTorch相关内容。

---
```
@@ -259,13 +264,25 @@ def train(hyp,  # path/to/hyp.yaml or hyp dictionary
     maps = np.zeros(nc)  # mAP per class
     results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
     scheduler.last_epoch = start_epoch - 1  # do not move
-    scaler = amp.GradScaler(enabled=cuda)
+    use_amp = (device.type == 'cuda' or opt.pyamp)
+    scaler = amp.GradScaler(enabled=use_amp)
+
     stopper = EarlyStopping(patience=opt.patience)
     compute_loss = ComputeLoss(model)  # init loss class
     LOGGER.info(f'Image sizes {imgsz} train, {imgsz} val\n'
                 f'Using {train_loader.num_workers} dataloader workers\n'
                 f"Logging results to {colorstr('bold', save_dir)}\n"
                 f'Starting training for {epochs} epochs...')
+
+    metric_collector = MetricCollector(
+        enable_only_benchmark=True,
+        record_elapsed_time=True,
+        record_hardware_time=True if opt.device == 'mlu' else False)
+    if opt.iters != -1:
+        epochs = start_epoch + math.ceil(opt.iters*1.0 / len(train_loader))
+        iters = opt.iters
+    metric_collector.place()
+
     for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
         model.train()
```
解析：
1. 添加MLU对自动混合精度（Automatic mixed precision，AMP）训练。

---
```
@@ -449,7 +475,7 @@ def parse_opt(known=False):
     parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
     parser.add_argument('--cache', type=str, nargs='?', const='ram', help='--cache images in "ram" (default) or "disk"')
     parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
-    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
+    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu; mlu')
     parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
     parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
     parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
@@ -465,6 +491,9 @@ def parse_opt(known=False):
     parser.add_argument('--freeze', type=int, default=0, help='Number of layers to freeze. backbone=10, all=24')
     parser.add_argument('--save-period', type=int, default=-1, help='Save checkpoint every x epochs (disabled if < 1)')
     parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')
+    parser.add_argument('--pyamp', action='store_true', help='using amp for mixed precision training')
+    parser.add_argument('--iters', type=int, default=-1, help="Total iters for benchmark.")
+    parser.add_argument('--skip', action='store_true', help='skip val or save pt.')
```
解析：
1. ```--device``` 运行参数增添‘mlu’选项。
2. ```--pyamp``` MLU训练支持自动混合精度训练，新增此参数设置。

---
```
@@ -501,16 +530,28 @@ def main(opt, callbacks=Callbacks()):
             opt.exist_ok, opt.resume = opt.resume, False  # pass resume to exist_ok and disable resume
         opt.save_dir = str(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))

+    if opt.device == 'mlu':
+        print(is_device_available)
+        assert is_device_available['mlu'], ""
+        device = torch.device('mlu')
+    else:
+        device = select_device(opt.device, batch_size=opt.batch_size)
```
解析：
1. 该步骤较为重要，表示当运行device选取为'mlu'时的配置选项。

## 4.4 训练

这里提供了三种运行方式：

1. Training: 基于数据从头开始训练；
2. From pretrained training：基于原始代码的模型文件进行训练；
3. Resume Training：在上次训练基础上继续训练。

### 1. Training

In [ ]:
# Train YOLOv5m on COCO for 1 epochs in mlu device
!cd yolov5 && python train.py --img 640 --batch 28 --epochs 1 --data  coco.yaml --weights "" --cfg yolov5m.yaml --device mlu

### 2. From pretrained training

In [ ]:
!cd yolov5 && python train.py --batch 28 --epochs 1 --data coco.yaml --cfg yolov5m.yaml  --device mlu  --weights ../yolov5_model/yolov5m.pt

### 3. Resume Training

!cd yolov5 && python train.py --resume --batch 28 --data coco.yaml --cfg yolov5m.yaml  --device mlu  --weights ./runs/train/exp3/weights/last.pt 

注意根据最终训练版本修改 --weights 的参数。本实验由于时间关系，故 Resume Training 注释方式提供

**参数说明：**

* imgsz: 训练、验证时图像大小，默认640；
* epoch: 训练迭代次数；
* batch：单个批次大小，若是运行中提示内存错误，可以考虑减少该数值；
* data: dataset.yaml 文件路径；
* weights: 初始化模型路径，若提示“No such file or directory”,注意查询对应文件夹是否有相应模型文件；
* cfg: model.yaml 路径；
* device: 运行设备选取，如 mlu，cuda 或 cpu； 
* resume: 在最近训练结果继续训练；
* 运行命令中可添加 `--pyamp` 参数进行自动混合精度训练；
* 更多参数设置及解析在 train.py 文件 parse_opt 函数中查看；
* 超参可在```/yolov5/data/hyps/typ.scratch.yaml```中设置。

## 4.5 精度验证

!cd yolov5 && python val.py --data coco.yaml --conf 0.001 --iou 0.65 --weight runs/train/exp4/weights/best.pt --device mlu

因为该文档和代码主要教大家如何在寒武纪MLU上运行 YOLOv5 的训练，其中迭代次数epoch设置较小，故精度验证以注释方式提供(若需要运行，建议测试 From pretrained training 保存的效果，并将上行代码改成 code 的形式)，只是验证该命令可用，不作实际精度测试。测试时注意根据上述训练模型保存情况修改 weight 的参数。

# 5 结语

从上述适配流程可知，采用 MLU370 进行 AI 模型训练流程与 GPU 使用较为一致，方便用户学习与扩展，极大降低了模型迁移成本和学习时间。同时采用 MLU370 训练能够加速模型的训练速度，与寒武纪MagicMind推理平台相配合使用，丰富训推一体平台的功能完善，让机器更好地理解与服务人类。  

## 5.1 回顾重点步骤
至此，基于寒武纪 MLU370 与 PyTorch 框架下的 YOLOv5 训练实验已经完毕。让我们回顾一下在使用寒武纪 MLU370 与 PyTorch 框架下，都有哪些主要开发步骤：
1. 新增MLU device支持，将模型与数据使用MLU进行训练。
2. 各种训练方式的使用，如采用预训练模型、finetune、resume以及自动混合精度的训练设置。
3. 使用MLU进行精度验证。

## 5.2 相关链接  

1. 对上述代码有疑问请提交ISSUE:  
https://gitee.com/cambricon/practices/issues  

2. 更多与寒武纪开发相关的有趣内容请移步至寒武纪开发者社区：    
https://developer.cambricon.com/

3. 如果有任何其他问题可到寒武纪开发者论坛提问，会有专人为您解答：  
https://forum.cambricon.com//list-1-1.html
